# Step 0: Who Am I? - Personal Identity Bootstrap

## The Story

You're a new security analyst joining the incident response team. Before you can investigate threats, create incidents, or collaborate with others, you need to **exist in the system**. But there's a problem—a circular dependency called the **bootstrap problem**.

To create your identity, you need:
- An email address (to communicate)
- A user account (to authenticate)
- An identity object (to link everything together)

But each of these references the others! How do we break the cycle?

**The solution**: Build from the ground up using **Pattern 3.2: Identity Sub-Pattern**.

## What You'll Learn

By the end of this notebook, you'll understand:
1. **The Bootstrap Problem**: Why identity creation requires careful ordering
2. **Pattern 3.2**: The Identity Sub-Pattern (Level 0 → Level 2)
3. **Dependency Hierarchy**: Building objects in the correct order
4. **Context Memory**: Where identities are stored for reuse

## The Characters

- **You** (TR_user): Your analyst identity - the primary character
- **Nick** (Team 1): Your first teammate
- **Hilda** (Team 2): Your second teammate  
- **Trusty** (Team 3): Your third teammate

Four identities, each following the same pattern. Learn it once, use it everywhere.

## The Journey

**Act 1**: Setup (get ready)
**Act 2**: Create YOUR identity (solve the bootstrap problem)
**Act 3**: Create TEAM identities (repeat the pattern)
**Act 4**: Summary (what we built)

Let's begin.

## Act 1: Setup - Getting Ready

### Scene 1: Import STIX Libraries

We need the STIX 2.1 standard to create cybersecurity objects. Think of STIX as the "language" we use to describe identities, threats, and incidents in a standardized way.

**What we're importing:**
- **Identity**: The main object representing a person or organization
- **EmailAddress**: Contact information (Level 0 - no dependencies)
- **UserAccount**: Login information (Level 0 - no dependencies)
- **IdentityContact**: Extension that adds phone numbers, social media, etc.

These are the building blocks. Let's load them.

In [ ]:
# Install STIX library
import sys
!{sys.executable} -m pip install stixorm

# Import STIX 2.1 objects
from stixorm.module.definitions.stix21 import Identity, EmailAddress, UserAccount

# Import OS Threat extensions for enhanced identities
from stixorm.module.definitions.os_threat import IdentityContact

print("✅ STIX 2.1 libraries loaded")
print("✅ Ready to create identity objects")


[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ STIX libraries initialized successfully
✅ OS Threat extensions loaded
✅ Logging configured


### Scene 2: Configure Python Path

We need access to the Brett Blocks utility functions. These are helper functions that make creating STIX objects easier.

Instead of writing 50 lines of code to create an identity, we'll use:
- `invoke_make_user_account_block()`
- `invoke_make_email_addr_block()`  
- `invoke_make_identity_block()`

Think of these as "shortcuts" that handle all the technical details for us.

In [ ]:
# Add parent directory to path for utility imports
import sys
sys.path.append('../')

print("✅ Python path configured")
print("✅ Utility functions now accessible")

✅ Working directory: c:\projects\brett_blocks\Orchestration
✅ Python path configured for Brett Blocks utilities


### Scene 3: Load Utilities and Data

Now we import the Brett Blocks "magic functions" that do the heavy lifting:

**Utility Functions:**
- `invoke_make_user_account_block()` - Creates user account objects
- `invoke_make_email_addr_block()` - Creates email address objects  
- `invoke_make_identity_block()` - Creates identity objects (links account + email)
- `invoke_save_user_context_block()` - Saves YOUR identity to `/usr/cache_me.json`
- `invoke_save_team_context_block()` - Saves TEAM identities to `/usr/cache_team.json`

**Data Configuration:**
We have 4 identities to create (you + 3 teammates). Each needs:
- Email address file (defines the email)
- Account file (defines the login)
- Identity file (defines the person)

The `identities_data` list tells us where to find these files for each person.

In [ ]:
# Import Brett Blocks utility functions
from Utilities.local_make_general import (
    invoke_save_user_context_block,
    invoke_save_team_context_block
)
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.local_make_sco import (
    invoke_make_user_account_block,
    invoke_make_email_addr_block
)

# Configure paths
path_base = "../Block_Families/StixORM/"
results_base = "../Orchestration/Results/"

# Define who we're creating (you + 3 teammates)
identities_data = [
    {
        "who": "me",
        "email": "SCO/Email_Addr/email_addr_TR_user.json",
        "acct": "SCO/User_Account/usr_account_TR_user.json",
        "ident": "SDO/Identity/identity_TR_user.json",
        "results": "step0/me"
    },
    {
        "who": "tea1",
        "email": "SCO/Email_Addr/email_addr_team1.json",
        "acct": "SCO/User_Account/usr_account_team1.json",
        "ident": "SDO/Identity/identity_team1.json",
        "results": "step0/nick1"
    },
    {
        "who": "tea2",
        "email": "SCO/Email_Addr/email_addr_team2.json",
        "acct": "SCO/User_Account/usr_account_team2.json",
        "ident": "SDO/Identity/identity_team2.json",
        "results": "step0/hilda2"
    },
    {
        "who": "tea3",
        "email": "SCO/Email_Addr/email_addr_team3.json",
        "acct": "SCO/User_Account/usr_account_team3.json",
        "ident": "SDO/Identity/identity_team3.json",
        "results": "step0/trusty3"
    }
]

print("✅ Utilities loaded successfully")
print(f"✅ Will create {len(identities_data)} identities:")
for ident in identities_data:
    print(f"   - {ident['who']}")
print("\n✅ Act 1 complete - Ready to create identities!")

✅ Brett Blocks utilities imported successfully
✅ Base paths configured:
   - Objects: ../Block_Families/StixORM/
   - Results: ../Orchestration/Results/
✅ Identity data loaded for 4 identities
   - me: step0/me
   - tea1: step0/nick1
   - tea2: step0/hilda2
   - tea3: step0/trusty3


## Act 2: Creating YOUR Identity - Solving the Bootstrap Problem

### The Challenge

We need to create 3 objects that reference each other:
- **user-account** (your login info)
- **email-addr** (your contact info)
- **identity** (your person, which references account + email)

But `identity` needs `user-account` and `email-addr` to exist first!

### The Solution: Pattern 3.2 (Identity Sub-Pattern)

We build in **dependency order**:

```
Level 0: user-account (no dependencies)
         email-addr (no dependencies)
         ↓
Level 2: identity (depends on both Level 0 objects)
```

**Why this works:**
- Level 0 objects don't reference anything else (they're "foundation")
- Level 2 identity references Level 0 objects (it's "built on top")
- We create bottom-up: Level 0 first, then Level 2

This is the **Identity Sub-Pattern** - you'll see it everywhere in STIX.

Let's build it step by step.

### Step 1: Find Your Data

First, let's locate the configuration files for YOUR identity (the "me" entry).

These files define:
- Your email address
- Your user account details
- Your identity information (name, role, etc.)

In [ ]:
# Find YOUR data configuration
for ident in identities_data:
    if ident["who"] == "me":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        break

print("✅ Your data located:")
print(f"   Email: {email_path}")
print(f"   Account: {acct_path}")
print(f"   Identity: {ident_path}")

✅ Personal identity data located:
   - Email config: SCO/Email_Addr/email_addr_TR_user.json
   - Account config: SCO/User_Account/usr_account_TR_user.json
   - Identity config: SDO/Identity/identity_TR_user.json
   - Results directory: step0/me


### Step 2: Create User Account (Level 0)

**First foundation piece**: Your user account.

This is **Level 0** because it has no embedded references - it's pure data:
- user_id
- account_login  
- display_name

**Why Level 0 first?**
Because the identity object (Level 2) will reference this via `x_stixorm_user_account_refs`.

**Pattern 3.2 in action**: Start at the bottom of the dependency hierarchy.

In [ ]:
# Create your user account (Level 0)
TR_user_acct = invoke_make_user_account_block(acct_path, results_path)

# Save to user context
acct_obj_path = results_base + results_path + "__usr_acct.json"
acct_context_path = results_base + "step0/context/TR__usr_acct_context.json"
result = invoke_save_user_context_block(acct_obj_path, acct_context_path)

print("✅ User account created (Level 0)")
print(f"   Type: {TR_user_acct['type']}")
print(f"   ID: {TR_user_acct['id']}")
print(f"   Saved to: /usr/cache_me.json")
print(f"\n📍 Level 0 foundation: 1 of 2 complete")

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
    "user_id": "79563902",
    "account_login": "tjones",
    "account_type": "soc,",
    "display_name": "Trusty Jones"
}
input data->{'stix_object': {'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--83658594-537d-5c32-b9f0-137354bd9bc3', 'user_id': '79563902', 'account_login': 'tjones', 'account_type': 'soc,', 'display_name': 'Trusty Jones'}}
✅ User account created and saved to user context
   - Object file: ../Orchestration/Results/step0/me__usr_acct.json
   - Context storage: ../Orchestration/Results/step0/context/TR__usr_acct_context.json
   - Storage result: {'context_result': 'User Directory ./generated/os-triage/context_mem/usr\nOptions context saved -> me\nstix_id -> user-account--83658594-537d-5c32-b9f0-137354bd9bc3'}


### Step 3: Create Email Address (Level 0)

**Second foundation piece**: Your email address.

This is also **Level 0** - no embedded references, just data:
- value (the email address itself)
- display_name

Notice we pass `TR_user_acct` to the function. This creates a **relationship** between email and account (stored separately as an SRO), but the email object itself remains Level 0.

**Pattern 3.2 continues**: Build the second Level 0 foundation object.

In [ ]:
# Create your email address (Level 0)
TR_email_addr = invoke_make_email_addr_block(email_path, results_path, TR_user_acct)

# Save to user context
email_obj_path = results_base + results_path + "__email.json"
email_context_path = results_base + "step0/context/TR__email_addr_context.json"
result = invoke_save_user_context_block(email_obj_path, email_context_path)

print("✅ Email address created (Level 0)")
print(f"   Type: {TR_email_addr['type']}")
print(f"   ID: {TR_email_addr['id']}")
print(f"   Saved to: /usr/cache_me.json")
print(f"\n📍 Level 0 foundation: 2 of 2 complete")
print(f"📍 Ready to build Level 2!")

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3",
    "value": "trusty@example.com",
    "display_name": "Trusty Jones",
    "belongs_to_ref": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3"
}
input data->{'stix_object': {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3', 'value': 'trusty@example.com', 'display_name': 'Trusty Jones', 'belongs_to_ref': 'user-account--83658594-537d-5c32-b9f0-137354bd9bc3'}}
✅ Email address created and linked to user account
   - Object file: ../Orchestration/Results/step0/me__email.json
   - Context storage: ../Orchestration/Results/step0/context/TR__email_addr_context.json
   - Storage result: {'context_result': 'User Directory ./generated/os-triage/context_mem/usr\nOptions context saved -> me\nstix_id -> email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3'}


### Step 4: Create Identity (Level 2)

**The culmination**: Your complete identity object.

This is **Level 2** because it has embedded references:
- `x_stixorm_user_account_refs` → points to your user account
- `x_stixorm_email_refs` → points to your email address

Both of those Level 0 objects now exist, so we can safely create the Level 2 identity that references them.

**Pattern 3.2 complete**: We've built the full Identity Sub-Pattern!

```
user-account (Level 0) ──┐
                          ├──→ identity (Level 2)
email-addr (Level 0) ─────┘
```

**This identity is YOU.** Every incident you investigate, every object you create, will reference this identity as the creator or observer.

In [ ]:
# Create your identity (Level 2) - links account + email
TR_ident = invoke_make_identity_block(
    ident_path, 
    results_path, 
    email_results=TR_email_addr, 
    acct_results=TR_user_acct
)

# Save to user context
ident_obj_path = results_base + results_path + "__ident.json"
ident_context_path = results_base + "step0/context/TR__ident_context.json"
result = invoke_save_user_context_block(ident_obj_path, ident_context_path)

print("✅ Identity created (Level 2)")
print(f"   Type: {TR_ident['type']}")
print(f"   ID: {TR_ident['id']}")
print(f"   Name: {TR_ident.get('name', 'N/A')}")
print(f"   Saved to: /usr/cache_me.json")
print(f"\n🎉 YOUR identity is complete!")
print(f"   ✅ user-account (Level 0)")
print(f"   ✅ email-addr (Level 0)")
print(f"   ✅ identity (Level 2)")
print(f"\n📍 Pattern 3.2 demonstrated successfully!")

type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'Trusty Jones', 'description': 'A Trusty Individual', 'roles': ['soc', 'support'], 'identity_class': 'individual', 'sectors': ['technology']}, 'extensions': {'extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498': {'extension_type': 'property-extension', 'contact_numbers': [], 'email_addresses': [], 'social_media_accounts': [], 'first_name': 'Me', 'last_name': 'Jones', 'middle_name': 'Percival', 'prefix': 'Dr', 'team': 'os-threat'}}, 'sub': {'contact_numbers': [{'contact_number_type': 'work-phone', 'contact_number': '0418-208-368'}], 'email_addresses': [{'digital_contact_type': 'work'}], 'social_media_accounts': [{'digital_contact_type': 'work'}]}},

## Act 3: Creating Team Identities - Pattern Reuse

### The Pattern Repeats

We just created YOUR identity using Pattern 3.2. Now we'll create 3 more identities for your teammates using **the exact same pattern**.

**This is the power of patterns**: Learn it once, use it everywhere.

For each teammate:
1. Create user-account (Level 0)
2. Create email-addr (Level 0)
3. Create identity (Level 2) linking both

The code is the same, only the input data changes (different names, emails, etc.).

**Why separate storage?**
- YOUR identity: `/usr/cache_me.json` (personal workspace)
- TEAM identities: `/usr/cache_team.json` (collaboration workspace)

This lets you quickly access your team without loading company data.

Let's create Nick, Hilda, and Trusty!

In [ ]:
# Create team member identities (Pattern 3.2 × 3)
team_count = 0

for ident in identities_data:
    # Skip "me" - only process team members
    if ident["who"][:3] != "tea":
        continue
    
    team_count += 1
    print(f"\n━━━ Creating Team Member {team_count} ━━━")
    
    # Extract paths
    email_path = ident["email"]
    acct_path = ident["acct"]
    ident_path = ident["ident"]
    results_path = ident["results"]
    
    # Define output paths
    acct_obj_path = results_base + results_path + "__usr_acct.json"
    acct_context_path = results_base + f"step0/context/usr_{results_path[-1:]}t__usr_acct_context.json"
    email_obj_path = results_base + results_path + "__email.json"
    email_context_path = results_base + f"step0/context/usr_{results_path[-1:]}t__email.json"
    ident_obj_path = results_base + results_path + "__ident.json"
    ident_context_path = results_base + f"step0/context/usr_{results_path[-1:]}t__ident_context.json"
    
    # Pattern 3.2: Level 0 → Level 0 → Level 2
    
    # Create user account (Level 0)
    user_acct = invoke_make_user_account_block(acct_path, results_path)
    invoke_save_team_context_block(acct_obj_path, acct_context_path)
    print(f"✅ user-account created (Level 0)")
    
    # Create email address (Level 0)
    user_email = invoke_make_email_addr_block(email_path, results_path, user_acct)
    invoke_save_team_context_block(email_obj_path, email_context_path)
    print(f"✅ email-addr created (Level 0)")
    
    # Create identity (Level 2)
    user_ident = invoke_make_identity_block(ident_path, results_path, 
                                           email_results=user_email, 
                                           acct_results=user_acct)
    invoke_save_team_context_block(ident_obj_path, ident_context_path)
    print(f"✅ identity created (Level 2)")
    print(f"   Name: {user_ident.get('name', 'N/A')}")
    print(f"   Saved to: /usr/cache_team.json")

print(f"\n🎉 Team complete! Created {team_count} team member identities")
print(f"📍 Pattern 3.2 used {team_count + 1} times total (you + team)")
print(f"\n✅ Act 3 complete!")


📝 Processing team member 1: tea1
{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--c58f2f3c-c3df-5f1a-9f19-e642a01bced3",
    "user_id": "79563903",
    "account_login": "hexpert",
    "account_type": "soc,",
    "display_name": "Helpful Expert"
}
input data->{'stix_object': {'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--c58f2f3c-c3df-5f1a-9f19-e642a01bced3', 'user_id': '79563903', 'account_login': 'hexpert', 'account_type': 'soc,', 'display_name': 'Helpful Expert'}}
   ✅ User account created and saved: {'context_result': 'Team Directory ./generated/os-triage/context_mem/usr\nOptions context saved -> team\nstix_id -> user-account--c58f2f3c-c3df-5f1a-9f19-e642a01bced3'}
{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--f4889f76-152d-5527-a85f-9e8c07bf71b9",
    "value": "helpful@example.com",
    "display_name": "Helpful Expert",
    "belongs_to_ref": "user-account--c58f2f3c-c3df-5f1a-9f19-e642a01bced3"
}

: 

## Act 4: Summary - What We Built and Why It Matters

### ✅ Objects Created

**Total**: 12 STIX objects (4 identities × 3 objects each)

**Your Identity:**
- 1× user-account (Level 0)
- 1× email-addr (Level 0)
- 1× identity (Level 2)

**Team Identities:**
- 3× user-account (Level 0)
- 3× email-addr (Level 0)
- 3× identity (Level 2)

### 📍 Pattern Demonstrated

**Pattern 3.2: Identity Sub-Pattern**

We used this pattern 4 times (once per person). Each time:
1. Create user-account (Level 0 - no dependencies)
2. Create email-addr (Level 0 - no dependencies)
3. Create identity (Level 2 - references both Level 0 objects)

**This is the foundation for ALL identities** - whether analyst, employee, company, or system.

### ?️ Context Memory Structure

```
context_mem/
└── usr/
    ├── cache_me.json        # YOUR identity (1 person)
    ├── cache_team.json      # TEAM identities (3 people)
    └── edges.json           # Relationships between objects
```

### 🔍 The Bootstrap Problem - Solved!

We broke the circular dependency by building in **dependency order**:
- Level 0 objects first (foundation - no embedded refs)
- Level 2 objects second (built on top - has embedded refs)

This is the **dependency hierarchy** you'll see throughout STIX:
- Can't create incidents without evidence (Level 6 needs Level 0-5)
- Can't create events without sightings (Level 4 needs Level 3)
- Can't create identities without accounts (Level 2 needs Level 0)

**Always build bottom-up.**

### 🎓 What You Learned

1. **Pattern 3.2 (Identity Sub-Pattern)**: The template for creating any identity
2. **Dependency Hierarchy**: Why order matters (Level 0 → Level 2)
3. **Context Memory**: Where identities are stored (`/usr/` for users)
4. **Pattern Reusability**: Same pattern works for you, team, company, systems
5. **Utility Functions**: How `invoke_make_*` functions simplify object creation

### 🚀 Next Steps

You're now ready for **Step 1: Company Setup**

In Step 1, you'll:
- Create company identity (Pattern 3.2 again!)
- Create employee identities (Pattern 3.2 × 15)
- Create system identities (Pattern 3.2 for servers, apps)
- Use IdentityContact extension (phone numbers, social media)

**Same pattern, bigger scale.**

### 💡 Why This Matters: Incident Reconstitution

Later, when you investigate incidents, you'll ask questions like:
- "Who created this incident?" → References YOUR identity
- "Who was the victim?" → References employee identity from Step 1
- "Who reported this?" → References team member identity

The graph preserves **provenance**: every action is attributed to an identity. This enables:
- **Audit trails**: Who did what, when?
- **Collaboration**: Work with your team
- **Attribution**: Tie evidence to witnesses
- **Access control**: What can this identity do?

**Your identity is the foundation for everything that comes next.**

### 🎉 Setup Complete!

You exist in the system. You can now create incidents, investigate threats, and collaborate with your team.

**Welcome to Brett Blocks.** 🚀